# Building a local DDF/kMS branch on the Bologna LOFAR cluster

### Comments? Questions? Complaints? Email etienne.bonnassieux@unibo.it

This notebook contains the commands necessary to build your own branch of DDF for dev work on the Bologna LOFAR clusters. It was created on 20/10/2020 and may not remain up-to-date forever. Note that, at the time of writing, it contains different build instructions than the documentation on the main DDFacet repository indicates: this is intentional.

## Building a local DDF

We begin by entering the dedicated singularity with all the prerequisite software properly installed, built and path'd. First:

In [ ]:
singularity-setup
lofar-setup d10.201020.09-ddf-private
lofar-singularity

The first two commands ensure you land in the correct singularity; the third puts you inside it. You can tell the difference by looking at your terminal: before, it should have looked something like 

[bonnassieux@lofar4 3C295_Frits]$

and it will now look like

Singularity> 

If so, things are working as they should. Now, we have to build the paths. Thankfully, the sysadmins have generously made this a simple task for us. Simply launch these commands:

In [ ]:
. /opt/lofar-cfg.sh
. /opt/lofar-init.sh

We are now in the proper environment, and can proceed to building DDFacet. Go to whever you want to have your local version installed; in my case, this will be /homes/bonnassieux

In [ ]:
cd /homes/bonnassieux
git clone https://github.com/cyriltasse/DDFacet
pip3 install --user -e DDFacet

At this point, you will download a bunch of prerequisites for the build. The program will fail upon downloading llvmlite: don't worry, that is normal and expected. The singularity setup will handle those builds. Once your pip3 install has failed (if it failed on llvmlite), proceed to build DDF:

In [ ]:
cd DDFacet
git checkout branchname
git submodule update --init --recursive
python3 setup.py build
python3 setup.py install --prefix=MyDDF

Note that the git clone command will prompt you to log into github. If you do not have access to the DDFacet github, you probably shouldn't be building a development branch of DDF for yourself in the first place. Contact your supervisor and/or whoever asked you to follow this tutorial for assistance.

This will create a version of DDF, from the branch branchname (replace with master, BeamATCA, BeamGMRT, SlitHugeIslands, etc as necessary for your specific case). When building, you will get a million warnings about unused variables: they have been forgotten by their creator and left to fend for themselves. As they do not care, neither should you.

After the build is complete, you can now call your own build of DDF locally. To do so, if you used the same prefix as given above (the actual string is arbitrary), call it as \$(INSTALLDIR)/DDFacet/\$(PREFIX)/bin/DDF.py. In my case, that means I call it as:

In [ ]:
/homes/bonnassieux/DDFacet/MyDDF/bin/DDF.py --version

For convenience, you can redefine an alias for this. I would personally recommend against this, to ensure you do not accidentally end up using the default DDF installed in /opt/DDFacet . Nevertheless,it can be done as:

In [ ]:
export alias DDF.py="/homes/bonnassieux/DDFacet/MyDDF/bin/DDF.py"
DDF.py --version

which should give the same output as above. Again, it is risky; do this at your own peril. 

## Changing the DDF branch

Now, say you've worked thus far in the MyCoolDDFBranch, but you need to use the awesome new features of the WowAmazing branch. To do this, simply return to wherever you cloned your DDF branch, which in my example case means:

In [ ]:
cd /homes/bonnassieux
cd DDFacet
git checkout WowAmazing
git submodule update --init --recursive
python3 setup.py build
python3 setup.py install --prefix=MyDDF

This will rebuild the python packages: if you run DDF.py --version, you will notice the version output changed. It is possible that your local DDF repository does not include the WowAmazing branch, if you've not been updating your local repo. This can be fixed with a git pull:

In [ ]:
git pull

at which point you can resume the steps above from checkout.

## Building a local killMS

If you've followed the advice given so far, you've built a python3 branch of DDFacet. However, you usually won't just want to do imaging - you'll want to do calibration, too. For this, you will need to build a python3 branch of killMS. The steps to do so will be outlined here, so you can have a beautiful fully-integrated, fully-operational pipeline of your very own.

As before, we begin by cloning and building, ideally in the same directory where we did this for DDFacet. We will have the same error with llvmlite as we had before: once again, this is expected and nothing to worry about.

In [ ]:
cd /homes/bonnassieux
git clone https://github.com/cyriltasse/killMS.git
cd killMS
git checkout MasterPython3
cd ..
pip3 install --user -e killMS

If you've already built and installed DDF, most if not all of the prerequisites will already be installed and ready to go, but it's always best to be sure. Once you run into this error:


Failed building wheel for llvmlite

you will need to follow the same steps as previously for DDF:

In [ ]:
cd killMS
python3 setup.py build
python3 setup.py install --prefix=MykMS

Once this is done, you should be able to call your custom killMS install without trouble, in my case using the following command:

In [ ]:
/homes/bonnassieux/killMS/MykMS/bin/kMS.py -h

which will simply give you the "documentation" for killMS. As before, I recommend building an alias for it, eg:

In [ ]:
export alias kMS.py="/homes/bonnassieux/killMS/MykMS/bin/kMS.py"

## Using your new DDF/kMS install

Now, say you built your awesome DDF branch inside the shiny dedicated singularity, but, undaunted, you decide to carry on and do some actual imaging with it. Fear not: other brave heroes have faced the same peril before you. For the sake of completeness, I recommend exiting your singularity first so that you have a clean break in your mind between the install and build process and the use.

First, go to your working directory. Then, load your singularity again:

In [ ]:
singularity-setup
lofar-setup d10.201020.09-ddf-private
lofar-singularity

Once inside your singularity, you have to do your init and config commands again. Importantly however, you also have to make sure your PYTHONPATH reads your custom DDF install and not the default /opt one! If you get an error that talks about 

File "/opt/SkyModel/Sky/ModVoronoi.py", line 145   
print "--"

then you simply have forgotten to do this PYTHONPATH step (what it tells you here is that it is loading the /opt SkyModel install, which is in python2). In short, as soon as you enter the singularity - and *every time* you enter the singularity - type the following commands:

In [ ]:
. /opt/lofar-cfg.sh
. /opt/lofar-init.sh
export PYTHONPATH="/homes/bonnassieux/DDFacet:/homes/bonnassieux/killMS:"$PYTHONPATH
alias kMS.py="/homes/bonnassieux/killMS/MykMS/bin/kMS.py"
alias DDF.py="/homes/bonnassieux/DDFacet/MyDDF/bin/DDF.py"

where I used /homes/bonnassieux/DDFacet because that is where I installed my custom DDFacet. Point to wherever you have installed yours. You should now be able to call your custom DDF as before, which in my case is:

In [ ]:
/homes/bonnassieux/DDFacet/MyDDF/bin/DDF.py -h

or using an alias if you created one. From that point on, you're good to go! Have fun.